This worksheet contains some examples of low-level Julia interprocess communication with an emphasis on when memory is allocated and when network communication happens.  This was done in version 0.3.11.

In [1]:
# Set up some convenient functions.

# Print the memory usage for julia processes.
function memstats()
  println(run(`ps -C julia -o pid,%mem,%cpu`))
end


# See what is defined on each worker.
function remote_vars()
  for w in workers()
    println(remotecall_fetch(w, whos))
  end
end

# This is like @everywhere but only runs on a particular process.
macro runat(p, ex)
  quote
    remotecall_fetch($p, ()->(eval(Main,$(Expr(:quote,ex))); nothing))
  end
end

# This is like @everywhere but for remote workers only.
macro everywhereelse(ex)
  quote
    @sync begin
      for w in workers()
        @async remotecall_fetch(w, ()->(eval(Main,$(Expr(:quote,ex))); nothing))
      end
    end
  end
end


# Define some objects to pass around that are big enough to show up in memstats().
mat = rand(int(1e4), int(1e4));
mat2 = rand(int(1e4), int(1e4));

# Remove remote workers to start.
for w in workers()
    if w != myid()
        rmprocs(w)
    end
end

memstats()

  PID %MEM %CPU
16809  1.1  0.1
23778 10.4  115
nothing


In [2]:
# Counterintuitively, `workers()` at first contains `[1]` but after adding workers does not.
println(workers())
addprocs(2)
println(workers())
memstats()

# Also: workers() does not necessarily return the same order everywhere.
@everywhere println(workers())

[1]
[2,3]
  PID %MEM %CPU
16809  1.1  0.1
23778 10.4 84.2
23790  0.5  174
23791  0.4  174
nothing
[2,3]
	From worker 3:	[3,2]
	From worker 2:	[2,3]


In [3]:
# A RemoteRef object has a where and whence field, but I'm not sure if they matter.

# A worker where and whence are the same:
pet = "aardvark"
rr = RemoteRef(myid())
println(rr)
put!(rr, pet)
@everywhereelse rr = remotecall_fetch(1, () -> rr)
@everywhereelse println(fetch(rr))

# This will hang if run.  You cannot put a new object into a RemoteRef which
# has already been passed to put!()
if false
    pet = "anteater"
    println("This will hang:")
    put!(rr, pet)
    println("...this line will never print.")
end

# Worker 3 seems to be able to put! to a RemoteRef that does not involve it at all: 
rr = RemoteRef(2)
println(rr)
@runat 3 begin
    lunch = "sandwich"
    rr = remotecall_fetch(1, () -> rr)
    put!(rr, lunch)
end
println(fetch(rr))


RemoteRef(1,1,10)
	From worker 3:	aardvark
	From worker 2:	aardvark
RemoteRef(2,1,15)
sandwich


In [ ]:
# Here we consider communicating `mat` to the workers.

# First, set up the remote references.
memstats()

# The RemoteRefs will be used to communicate data from worker 1 to the other workers
@everywhere rr = RemoteRef(1)
rr_array = [ remotecall_fetch(worker, () -> rr) for worker in workers() ]

# Next, put `mat` into each remote reference.
for worker_rr in rr_array
  put!(worker_rr, mat)
end

# Remote memory for `mat` has not been allocated yet.
memstats()


In [ ]:
# We can also `fetch()` without allocating memory:
for worker_rr in rr_array
  println(worker_rr.where, ": ", fetch(worker_rr)[1:5, 1:5])
end
memstats()

In [ ]:
# However, assigning actually allocats memory.
# Note that extra memory is also allocated no the master during the copy but is freed by gc().
fetch_time1 = time()
@everywhereelse mat11 = fetch(rr)[1,1]
fetch_time1 = time() - fetch_time1
memstats()
gc()
memstats()


In [ ]:
# Subsequent fetches take just as much time and allocate just as much memory
# before gc().  It is evidently re-fetching each time.
fetch_time2 = time()
@everywhereelse mat12 = fetch(rr)[1,2]
fetch_time2 = time() - fetch_time2
memstats()

println("Fetch time 1: ", fetch_time1)
println("Fetch time 2: ", fetch_time2)

In [ ]:
# Make a local copy of the whole array.  The fetch time is again the same.
fetch_time3 = time()
@everywhereelse mat_worker = fetch(rr)
fetch_time3 = time() - fetch_time3
memstats()

# Again, the extra memory is freed with garbage collection.
@everywhere gc()
memstats()
println("Fetch time 3: ", fetch_time3)

In [ ]:
# Confirm that the copy on worker two is in fact a local copy by changing some values.
@runat 2 mat_worker[1,1]= -20.
@runat 2 rr2 = RemoteRef(2)
@runat 2 put!(rr2, mat_worker)
rr2 = remotecall_fetch(2, () -> rr2)
mat_worker = fetch(rr2);
println("Value from remote worker: ", mat_worker[1, 1])
println("Original value: ", mat[1, 1])

In [ ]:
# As expected, allocating memory on process 2 only runs up the memory usage on process 2.
memstats()
@runat 2 num_big_mats = 5
@runat 2 big_mats = Array(Any, num_big_mats);
@runat 2 for i = 1:num_big_mats
            big_mats[i] = rand(int(1e4), int(1e4))
          end
memstats()

In [ ]:
# Subsequent fetches doesn't pick up changes in the immutable objects
@runat 2 rr = RemoteRef(1)
rr = remotecall_fetch(2, () -> rr)
myx = 1.23
put!(rr, myx)
@runat 2 println(fetch(rr))
myx += 1.45
@runat 2 println(fetch(rr))

# But it does in mutable ones
@runat 2 rr = RemoteRef(1)
rr = remotecall_fetch(2, () -> rr)
myxvec = [ 1.23 ]
put!(rr, myxvec)
@runat 2 println(fetch(rr))
myxvec[1] += 1.45
@runat 2 println(fetch(rr))

In [ ]:
# Note that @spawnat doesn't work for assignment since it resolves variables on the
# calling process.
for w in workers()
  @spawnat w y = 1.234
end
println("After running @spawnat:")
remote_vars()

# This works though
@everywhere y = 1.345
println("After running @everywhere:")
remote_vars()